## In this notebook we tried to build our own architecture for the CNN

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/prova_NN

In [ ]:
import os
import tensorflow as tf
tfk = tf.keras
tfkl = tf.keras.layers
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tqdm.notebook import tqdm
import cv2

print(tf.__version__)

# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Grid-search for architecture

In [ ]:
dataset_dir = 'training'

In [ ]:
input_shape = (256, 256, 3)
epochs = 5
img_height = 256 
img_width = 256
batch_size = 64

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale=1/255.,
                                    validation_split=0.1)

train_gen = train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=64,
                                               shuffle=True,
                                               seed=seed,
                                               subset='training'
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=dataset_dir,
                                               target_size=(256,256),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=64,
                                               shuffle=False,
                                               seed=seed,
                                               subset='validation'
                                               )


In [ ]:
from datetime import datetime
def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('architecture')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  tb_callback = tf.keras.callbacks.TensorBoard(
      log_dir = tb_dir,
      profile_batch = 0,
      
  )
  callbacks.append(tb_callback)

  return callbacks

In [ ]:
conv_layers = [3,4,5,6]
dense_layers = [0,1,2]

for dense_layer in dense_layers:
    for conv_layer in conv_layers:
        name = "{}-conv-{}-dense-{}".format(conv_layer,dense_layer,int(time.time()))
        callback = create_folders_and_callbacks(name)
        
        model = tfk.Sequential()
        
        model.add(tfkl.Conv2D(filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)))

        model.add(tfkl.MaxPooling2D(pool_size = (2, 2)))

        for l in range(conv_layer -2 ):
            model.add(tfkl.Conv2D(filters=16**(l+1),
                    kernel_size=(3, 3),
                    strides = (1, 1),
                    padding = 'same',
                    activation = 'relu',
                    kernel_initializer = tfk.initializers.GlorotUniform(seed)))

            model.add(tfkl.MaxPooling2D(pool_size = (2, 2)))

        model.add(tfkl.Flatten())
        model.add(tfkl.Dropout(0.3,seed = seed))

        for l in range(dense_layer):
            model.add(tfkl.Dense(256/(2**l),'relu',kernel_initializer=tfk.initializers.GlorotUniform(seed)))
            model.add(tfkl.Dropout(0.3,seed = seed))
        
        model.add(tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed)))

        model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
        

        history = model.fit( x = train_gen,
                            epochs = epochs,
                            validation_data = valid_gen,
                            callbacks = callback).history


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir architecture

the result of the gridsearch were unsurprising, the bigger the model te better the performance, so we decided to go for a middle of the road model 

we then trained a model with 4 conv layers and input of 128 to reduce the training time first with no data augmentation to get a starting point

In [ ]:
dataset_dir = 'training'

In [ ]:
input_shape = (128, 128, 3)
epochs = 8
img_height = 128
img_width = 128
batch_size = 64

In [ ]:
train_data_gen = ImageDataGenerator(
    rescale = 1/255.,
    validation_split = 0.2,
)

train_gen =  train_data_gen.flow_from_directory(
    directory = dataset_dir,
    target_size = (128,128),
    color_mode = 'rgb',
    classes = None,
    class_mode = 'categorical',
    batch_size = 64,
    shuffle = True,
    seed = seed,
    subset = 'training'
)

val_gen = train_data_gen.flow_from_directory(
    directory = dataset_dir,
    target_size = (128,128),
    color_mode = 'rgb',
    classes = None,
    class_mode = 'categorical',
    batch_size = 64,
    shuffle = True,
    seed = seed,
    subset = 'validation'
)

In [ ]:
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=(128,128,3), name='Input')
    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool4)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=256, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.3, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
model = build_model(input_shape)
model.summary()

In [ ]:
history = model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data =val_gen,
    callbacks = callbacks,
).history

In [ ]:
model.save('trained_no_aug_model')

we then trained the model with data augmentation

In [ ]:
input_shape = (128, 128, 3)
epochs = 10
img_height = 128
img_width = 128
batch_size = 64
validation_split = 0.15

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug_train_data_gen = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = 0.3,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'constant',
    cval = 0,
    rescale = 1/255.,
    validation_split = 0.2,
)

aug_val_data_gen = ImageDataGenerator(
    rescale = 1/255.,
    validation_split = 0.2,
)

aug_train_gen =  aug_train_data_gen.flow_from_directory(
    directory = dataset_dir,
    target_size = (128,128),
    color_mode = 'rgb',
    classes = None,
    class_mode = 'categorical',
    batch_size = 64,
    shuffle = True,
    seed = seed,
    subset = 'training'
)

val_train_gen = aug_val_data_gen.flow_from_directory(
    directory = dataset_dir,
    target_size = (128,128),
    color_mode = 'rgb',
    classes = None,
    class_mode = 'categorical',
    batch_size = 64,
    shuffle = True,
    seed = seed,
    subset = 'validation'
)

In [ ]:
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tfk.callbacks.ModelCheckpoint(
      filepath = os.path.join(ckpt_dir,'cp.ckpt'),
      save_weights_only=False, 
      save_best_only = False
  )
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs') 
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  tb_callback = tf.keras.callbacks.TensorBoard(
      log_dir = tb_dir,
      profile_batch = 0,
      
  )
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [ ]:
model =  tfk.models.load_model('trained_no_aug_model')

In [ ]:
callbacks = create_folders_and_callbacks(model_name='data_aug')

In [ ]:
history = model.fit(
    x = aug_train_gen,
    epochs = 20,
    validation_data =val_train_gen,
    callbacks = callbacks,
).history

In [ ]:
model.save("data_augmentation/Aug_Best")

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir data_augmentation/Aug_Best/

we then dropped the learning rate

In [ ]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
history5 = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data =val_train_gen,
    callbacks = callbacks,
).history

In [ ]:
model.save("data_augmentation/Aug_Best2")

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir data_augmentation/Aug_Best2/

this model was tested and got 50% accuracy

we then decided to increase the network size, and used an unbalanced dataset and a custom preprocessing function to increase performance

In [ ]:
epochs = 10
img_height = 224
img_width = 224
batch_size = 32
validation_split = 0.15
input_shape = (img_height, img_width, 3)

In [ ]:
def build_model2(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')
    conv1 = tfkl.Conv2D(
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv1)

    conv2 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv2)

    conv3 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(conv5)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool5)
    flattening_layer = tfkl.Dropout(0.3, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=256, name='Classifier', kernel_initializer=tfk.initializers.GlorotUniform(seed), activation='relu')(flattening_layer)
    classifier_layer = tfkl.Dropout(0.3, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=14, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
model2 = build_model2(input_shape)
model2.summary()

In [ ]:
callbacks = create_folders_and_callbacks(model_name = 'data_aug_enlarged')

In [ ]:
def custom_preprocess(image, probs = [0.4, 0.2, 0.2, 0.3]):
    """ probs are a list of length 3
      prob[0] constrols the noise added to the black background
      prob[1] constrols swapping of color channels
      prob[2] constrols HSV hue of the image
      prob[3] constrols blurring of image
      """
      # see https://stackoverflow.com/questions/57265893/change-colors-with-imagedatagenerator
    # Generate random values
    A,B,C,D = np.random.rand(4)
    
    # Define propabilites for each of the three augmentations
    if len(probs) != 4:
        raise ValueError("Lenght of threshold should be 3")
    else:
        thresholds = probs
        
    
    # Adds noise in the black background
    if A <= thresholds[0]:
        BACKGROUND_VALUE = 0 # Here I assume that 0 is the background colour
        size = image[image==BACKGROUND_VALUE].shape[0] 
        values = np.random.uniform(low=image.min(), high=image.max(), size=(size,))
        image[image==0] = values
    
    # Swap color channels
    if B <= thresholds[1]:
        dims = np.arange(3)
        np.random.shuffle(dims)
        image = image[...,[dims[0],dims[1],dims[2]]]
    
    # Change the hue of the image
    if C <= thresholds[2]:
        image = np.uint8(np.array(image))
        image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
        image = image.astype(np.float32)
        
    # Blurs the image slightly
    if D <= thresholds[3]:
        image = cv2.blur(image,(5,5))
    return image

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug_train_data_gen = ImageDataGenerator(validation_split=validation_split,
    rotation_range = 45,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    brightness_range = [0.3,1.3],
    shear_range = 10,
    zoom_range = [0.3,1.2],
    channel_shift_range = 40,
    horizontal_flip  = True,
    vertical_flip = True,
    preprocessing_function = custom_preprocess,
)

aug_val_data_gen = ImageDataGenerator(
    validation_split = validation_split,
)

aug_train_gen =  aug_train_data_gen.flow_from_directory(
    directory = dataset_dir,
    target_size = (img_height,img_width),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
    seed = seed,
    subset = 'training'
)

val_train_gen = aug_val_data_gen.flow_from_directory(
    directory = dataset_dir,
    target_size = (img_height,img_width),
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
    seed = seed,
    subset = 'validation'
)

In [ ]:
history = model2.fit(
    x = aug_train_gen,
    epochs = 20,
    validation_data = val_train_gen
).history

In [ ]:
model2.save('enlarged')

this model reached 60% accuracy on the test set

# Global averaging pooling experiments

we then decided to replace the flattening at the end of the convolutional part of the network with a global averaging polling layer 
we used the balanced dataset and the same preprocessing function as above

In [ ]:
def build_model3(input_shape):
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'Conv1')(input_layer)
    pool1 = tfkl.MaxPooling2D(name = 'Pool1')(conv1)

    conv2 = tfkl.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'Conv2')(pool1)
    pool2 = tfkl.MaxPooling2D(name = 'Pool2')(conv2)

    conv3 = tfkl.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'Conv3')(pool2)
    pool3 = tfkl.MaxPooling2D(name = 'Pool3')(conv3)

    conv4 = tfkl.Conv2D(
        filters=256,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'Conv4')(pool3)
    pool4 = tfkl.MaxPooling2D(name = 'Pool4')(conv4)

    conv5 = tfkl.Conv2D(
        filters=512,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'Conv5')(pool4)
    pool5 = tfkl.MaxPooling2D(name = 'Pool5')(conv5)

    glob_pooling = tfkl.GlobalAveragePooling2D(name = 'Globalpooling')(pool5)
    glob_pooling = tfkl.Dropout(0.3, seed=seed, name='GloablPoolingDropout')(glob_pooling)

    classifier_layer = tfkl.Dense(
        units=128,  
        activation='relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name='Classifier')(glob_pooling)
    classifier_layer = tfkl.Dropout(0.3, seed=seed, name='ClassifierDropout')(classifier_layer)

    output_layer = tfkl.Dense(
        units=14, 
        activation='softmax', 
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name='Output')(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='gap_model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [ ]:
model = build_model3(input_shape)
model.summary()

In [ ]:
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = val_train_gen,
).history


In [ ]:
model = tfk.models.load_model('gap_aug1')

In [ ]:
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = val_train_gen,
).history

In [ ]:
model.save('gap_aug2')

In [ ]:
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-4), metrics='accuracy')

In [ ]:
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = val_train_gen,
).history

In [ ]:
model.save('gap_aug3')

this model acheived a 75% score on the test set, which is our best score for a model not based on transfer learning and fine tuning

Since this model is fully convolutional thanks to the GAP layer I can increase the input size using the same weights. I now build the same model but increase the input shape and transfer the weights trained above

In [ ]:
model2 = build_model3((300,300,3))
model2.summary()

In [ ]:
model2.load_weights('gap_aug3')

In [ ]:
model2.save('gap_model_large_input')

this increased the performance by arround 2% on the test data

we then moved to transfer learning and fine tuning to reach abetter score 